# Scrape Parliament Members

## Import Libraries

In [5]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

In [301]:
url = "http://www.singapore-elections.com/"

page = requests.get(url)

In [302]:
page.encoding

'ISO-8859-1'

In [303]:
bs= BeautifulSoup(page.text,'html')

In [304]:
candidate_pages = bs.find_all('a',{'href': re.compile(r'candidates/')})

In [305]:
candidate_pages[0]['href']

'candidates/a-b.html'

In [306]:
uri = candidate_pages[0]['href']
src = "http://www.singapore-elections.com/"
page = src + uri
res = requests.get(page)
c  = BeautifulSoup(res.text,'lxml')

In [307]:
candidate_row = c.find('table').find_all('table')[1].find_all('tr')

In [308]:
candidate_row[-6].get_text()

'\n\n\n\n\xa0\n\xa0Elected MP\n\xa0\n\xa0Non-Constituency MP\n\xa0\n\xa0Forfeited deposit\n\n\n\n'

In [309]:
candidate_row[0].find('img')['title']

"People's Action Party"

In [310]:
def get_row_info(row):
    name = row.find('td').get_text()
    entity = {
        'name' :  name,
        'results' : []
    }
    details = row.find_all('td')
    keys = []
    for key,result in enumerate(details[1:]):
        try:
            ge_info = str(result).split('<br/>')
            party = result.find('img')['title']
            detail = {}
            detail['id'] = key
            keys.append(key)
            detail['party'] = party
            detail['GE_DATE'] = BeautifulSoup(ge_info[1],'lxml').get_text()
            detail['Consituency'] = BeautifulSoup(ge_info[2],'lxml').get_text()
            detail['Margin'] = BeautifulSoup(ge_info[3],'lxml').get_text()
            entity['results'].append(detail)
        except TypeError:
            pass
        except KeyError:
            party = BeautifulSoup(ge_info[2],'lxml').get_text()
            detail = {}
            detail['id'] = key
            keys.append(key)
            detail['party'] = party
            detail['GE_DATE'] = BeautifulSoup(ge_info[1],'lxml').get_text()
            detail['Consituency'] = BeautifulSoup(ge_info[2],'lxml').get_text()
            detail['Margin'] = BeautifulSoup(ge_info[3],'lxml').get_text()
            entity['results'].append(detail)
            continue

    return max(keys) , entity

In [311]:
get_row_info(candidate_row[5])


(1,
 {'name': 'ABDUL KARIM bin Abdul Sattar',
  'results': [{'id': 0,
    'party': 'Pertubuhan Kebangsaan Melayu Singapura',
    'GE_DATE': 'GE 1988',
    'Consituency': 'Buona Vista',
    'Margin': '3.1%'},
   {'id': 1,
    'party': 'Pertubuhan Kebangsaan Melayu Singapura',
    'GE_DATE': 'GE 1991',
    'Consituency': 'Buona Vista',
    'Margin': '20.6%'}]})

In [312]:
def parse_page(candidate_rows, data):
    prev_idx = 0
    idx = 0
    for candidate in candidate_rows:
        name = candidate.find('td').get_text()
#         print(f"{idx} : {name.strip()}")
        if "Elected MP" not in name.strip():
#             print('true')
            if not re.match('GE',name):
#                 print('individual')
                max_keys, info = get_row_info(candidate)
                data.append(info)
                prev_name = name
                _, prev_info = get_row_info(candidate)
                prev_idx = idx
                prev_key_idx = max_keys
#                 print(max_keys)
#                 print(f"Previous Index : {prev_idx}")
                idx += 1
            else:
#                 print('merged')
                details = candidate.find_all('td')
#                 print(details)
                key = max_keys + 1
#                 print(key)
#                 print(f"Previous Index : {prev_idx}")
                for result in details:
                    if result.get_text().strip() != "":
#                         print(f"Previous Index : {prev_idx}")
#                         print(key,result.get_text())
                        try:
                            ge_info = str(result).split('<br/>')
#                             print(BeautifulSoup(ge_info[1],'lxml').get_text())
                            party = result.find('img')['title']
                            detail = {}
                            detail['id'] = key 
                            detail['party'] = party
                            detail['GE_DATE'] = BeautifulSoup(ge_info[1],'lxml').get_text()
                            detail['Consituency'] = BeautifulSoup(ge_info[2],'lxml').get_text()
                            detail['Margin'] = BeautifulSoup(ge_info[3],'lxml').get_text()
                            data[prev_idx]['results'].append(detail)
#                             print(data[prev_idx]['results'])
                        except TypeError:
                            pass
                        except IndexError:
                            pass 
                        except KeyError:
                            party = BeautifulSoup(ge_info[2],'lxml').get_text()
                            detail = {}
                            detail['id'] = key
                            detail['party'] = party
                            detail['GE_DATE'] = BeautifulSoup(ge_info[1],'lxml').get_text()
                            detail['Consituency'] = BeautifulSoup(ge_info[2],'lxml').get_text()
                            detail['Margin'] = BeautifulSoup(ge_info[3],'lxml').get_text()
                            data[prev_idx]['results'].append(detail)
                            continue
                        
                        key += 1
                    else:
                        pass
                max_keys = key - 1
        else:
            break
    return data

In [313]:
src = "http://www.singapore-elections.com/"
info = []

for row in tqdm(candidate_pages):
    page_info = []
    page = src + row['href']
    res = requests.get(page)
    c  = BeautifulSoup(res.text,'lxml')
    rows = c.find('table').find_all('table')[1].find_all('tr')
    info.extend(parse_page(rows, page_info))



  0%|          | 0/16 [00:00<?, ?it/s]

  6%|▋         | 1/16 [00:01<00:25,  1.67s/it]

 12%|█▎        | 2/16 [00:03<00:23,  1.71s/it]

 19%|█▉        | 3/16 [00:04<00:18,  1.43s/it]

 25%|██▌       | 4/16 [00:05<00:15,  1.27s/it]

 31%|███▏      | 5/16 [00:06<00:14,  1.28s/it]

 38%|███▊      | 6/16 [00:07<00:11,  1.19s/it]

 44%|████▍     | 7/16 [00:09<00:11,  1.31s/it]

 50%|█████     | 8/16 [00:10<00:10,  1.37s/it]

 56%|█████▋    | 9/16 [00:11<00:08,  1.21s/it]

 62%|██████▎   | 10/16 [00:11<00:06,  1.04s/it]

 69%|██████▉   | 11/16 [00:12<00:04,  1.09it/s]

 75%|███████▌  | 12/16 [00:13<00:03,  1.12it/s]

 81%|████████▏ | 13/16 [00:15<00:03,  1.20s/it]

 88%|████████▊ | 14/16 [00:17<00:02,  1.43s/it]

 94%|█████████▍| 15/16 [00:18<00:01,  1.30s/it]

100%|██████████| 16/16 [00:19<00:00,  1.24s/it]

In [248]:
info[3]

{'name': 'ABDUL HAMID bin Jumat',
 'results': [{'id': 0,
   'party': 'United Malays National Organisation',
   'GE_DATE': 'GE 1955',
   'Consituency': 'Ulu Bedok',
   'Margin': '61.0%'},
  {'id': 1,
   'party': 'United Malays National Organisation',
   'GE_DATE': 'GE 1959',
   'Consituency': 'Geylang Serai',
   'Margin': '63.1%'}]}

In [246]:
candidate_pages

[<a href="candidates/a-b.html">A - B</a>,
 <a href="candidates/c.html">C</a>,
 <a href="candidates/d-f.html">D - F</a>,
 <a href="candidates/g.html">G</a>,
 <a href="candidates/h-i.html">H - I</a>,
 <a href="candidates/j-k.html">J - K</a>,
 <a href="candidates/la-le.html">La - Le</a>,
 <a href="candidates/li-lu.html">Li - Lu</a>,
 <a href="candidates/m.html">M</a>,
 <a href="candidates/n.html">N</a>,
 <a href="candidates/o.html">O</a>,
 <a href="candidates/p-r.html">P - R</a>,
 <a href="candidates/s.html">S</a>,
 <a href="candidates/t.html">T</a>,
 <a href="candidates/u-w.html">U - W</a>,
 <a href="candidates/y-z.html">Y - Z</a>]

## Data Consolidation

In [446]:
import pandas as pd
import numpy as np

template = pd.DataFrame(columns=["Name","Party","GE","Constituency","Margin"])
for person in tqdm(info):
    name = person['name']
    for result in person['results']:
        detail = {
            'Name' : name,
            'Party' : result['party'],
            'GE' : result['GE_DATE'],
            'Constituency' : result ['Consituency'],
            'Margin' : result['Margin']
        }
        template = template.append(detail,ignore_index=True)
        
                




  0%|          | 0/1135 [00:00<?, ?it/s]


  1%|          | 9/1135 [00:00<00:12, 87.92it/s]


  2%|▏         | 22/1135 [00:00<00:11, 96.83it/s]


  4%|▎         | 42/1135 [00:00<00:09, 114.36it/s]


  5%|▌         | 59/1135 [00:00<00:08, 126.59it/s]


  6%|▋         | 71/1135 [00:00<00:08, 123.96it/s]


  7%|▋         | 85/1135 [00:00<00:08, 127.32it/s]


  9%|▊         | 99/1135 [00:00<00:07, 130.33it/s]


 10%|▉         | 112/1135 [00:00<00:08, 126.61it/s]


 11%|█▏        | 129/1135 [00:00<00:07, 135.40it/s]


 13%|█▎        | 143/1135 [00:01<00:07, 128.05it/s]


 14%|█▍        | 161/1135 [00:01<00:07, 132.12it/s]


 15%|█▌        | 175/1135 [00:01<00:08, 107.12it/s]


 17%|█▋        | 189/1135 [00:01<00:08, 114.21it/s]


 18%|█▊        | 205/1135 [00:01<00:07, 124.25it/s]


 19%|█▉        | 219/1135 [00:01<00:07, 127.51it/s]


 21%|██        | 235/1135 [00:01<00:06, 135.15it/s]


 22%|██▏       | 250/1135 [00:01<00:07, 119.95it/s]


 24%|██▍       | 270/1135 [00:02<00:06, 135.75

## Data Cleaning

In [447]:
template.head()

,Name,Party,GE,Constituency,Margin
0,ABBAS bin Abu Amin,People's Action Party,GE 1980,Pasir Panjang,72.6%
1,ABBAS bin Abu Amin,People's Action Party,GE 1984,Pasir Panjang,59.3%
2,ABBAS bin Abu Amin,People's Action Party,GE 1988,Pasir Panjang GRC,61.6%
3,ABDUL AZIZ bin Karim,People's Action Party,GE 1968,Kallang,Uncontested
4,ABDUL AZIZ bin Karim,People's Action Party,GE 1972,Kallang,79.8%


In [448]:
template.iloc[542]

Name            GOH Keow Wah (Gilbert)Îâ¼ÒºÍ
Party                           Reform Party
GE                                   GE 2015
Constituency                  Ang Mo Kio GRC
Margin                                 21.4%
Name: 542, dtype: object

In [449]:
re.findall('\d+',template.iloc[541].GE)[0]

'2011'

In [450]:
# Get English Names Only
template.Name = template.Name.map(lambda x : re.sub(r'[^\x00-\x7F]+','',x))
# Map to Election Type
template['election_type'] = template.GE.map(lambda x : re.sub('\d+','',x).strip())
template['election_year'] = template.GE.map(lambda x : int(re.findall('\d+',x)[0]))

In [451]:
template.iloc[542]

Name             GOH Keow Wah (Gilbert)
Party                      Reform Party
GE                              GE 2015
Constituency             Ang Mo Kio GRC
Margin                            21.4%
election_type                        GE
election_year                      2015
Name: 542, dtype: object

We want to filter the list such that the most recent election year will remain in the row. This is to allow us to instantiate the electionDB.

In [452]:
template.election_year = template.election_year.map(lambda x : pd.to_datetime(x,format='%Y'))

In [461]:
template.head()

,Name,Party,GE,Constituency,Margin,election_type,election_year
0,ABBAS bin Abu Amin,People's Action Party,GE 1980,Pasir Panjang,72.6%,GE,1980-01-01
1,ABBAS bin Abu Amin,People's Action Party,GE 1984,Pasir Panjang,59.3%,GE,1984-01-01
2,ABBAS bin Abu Amin,People's Action Party,GE 1988,Pasir Panjang GRC,61.6%,GE,1988-01-01
3,ABDUL AZIZ bin Karim,People's Action Party,GE 1968,Kallang,Uncontested,GE,1968-01-01
4,ABDUL AZIZ bin Karim,People's Action Party,GE 1972,Kallang,79.8%,GE,1972-01-01


In [463]:
recent_df = template.groupby('Name').last()

### Get only names

In [469]:
recent_df = recent_df.reset_index()

In [470]:
len(recent_df.Name.unique())

1133

## Data Export

In [393]:
template.to_csv('party.csv')

## Rev 2 : Getting More Info On Current Members

In [279]:
import unicodedata
import re

src = "https://www.parliament.gov.sg/mps/list-of-current-mps/mp/details/amrin-amin"
r = requests.get(src)
bs = BeautifulSoup(r.text,'html5lib')

In [280]:
name = bs.find('h1',{'class':'indv-mp-header-mobile'}).get_text()
name

'Mr Amrin Amin'

In [323]:
party = bs.find('div',{'class':'row mp-party-wrap'}).find('div',{'class':"col-md-9 col-xs-12"}).get_text().strip()
party

'People’s Action Party'

In [281]:
designation = bs.find('div',{'class':'row mp-designation-wrap'}).find('div',{'class':"col-md-9 col-xs-12"}).get_text().split('\t',1)[0].replace('\n','')
designation

'Senior Parliamentary Secretary, Ministry of Home Affairs & Ministry of Health'

In [282]:
year_birth = re.sub('\n|\t','',bs.find_all('div',{'class':'row mp-party-wrap'})[1].find('div',{'class':'col-md-9 col-xs-12'}).get_text())
year_birth

'1978'

In [293]:
constituency = bs.find('div', {'class':'row mp-constituency-wrap'})\
                .find('div',{'class':"col-md-9 col-xs-12"}).get_text().split('\n')[1].strip()
constituency

'Sembawang GRC'

In [107]:
appts = bs.find('div',{'class':'indv-mp-info'})
appts_clean = re.sub('(<!-- .* -->)','<seperator>',str(appts))

In [153]:
split_appts = appts_clean.split('<seperator>')
appts.find_all('h2',{'class':'indv-mp-subheader'})

[<h2 class="indv-mp-subheader">Office-Holding Appointments</h2>,
 <h2 class="indv-mp-subheader">Member of Parliament</h2>,
 <h2 class="indv-mp-subheader">Select Committees</h2>,
 <h2 class="indv-mp-subheader">Meet the People Session</h2>]

In [406]:
for item in split_appts:
    if'Office-Holding Appointments' in item:
    #OFFICE HOLDING APPOINTMENTS
        office_appts = item
        office_appts_list = BeautifulSoup(office_appts,'html').find_all('div', {'class':'row mp-info'})
        o_appts = {}
        for key,item in enumerate(office_appts_list):
            pos_info = {}
            pos_info_raw = re.sub('\n\n','',re.sub('\n\n\n',' : ', item.get_text()))
            pos_info['duration'] = pos_info_raw.split(':')[0].strip()
            pos_info['position'] = pos_info_raw.split(':')[1].strip()
            o_appts[key] = pos_info
    if'Member of Parliament' in item:
    #OFFICE HOLDING APPOINTMENTS
        mp_appts = item
        mp_appts_list = BeautifulSoup(mp_appts,'html').find_all('div', {'class':'row mp-info'})
        m_appts = {}
        for key,item in enumerate(mp_appts_list):
            mp_info = {}
            mp_info_raw = re.sub('\n\n','',re.sub('\n\n\n',' : ', item.get_text()))
            mp_info['duration'] = mp_info_raw.split(':')[0].strip()
            mp_info['position'] = mp_info_raw.split(':')[1].strip()
            m_appts[key] = mp_info 
    if 'Select Committees' in item:
        select_list = BeautifulSoup(select,'html').find_all('div', {'class':'row mp-info'})
        select_appts = {}
        for key,item in enumerate(select_list):
            select_info = {}
            select_info_raw = re.sub('\n\n','',re.sub('\n\n\n',' : ', item.get_text()))
            select_info['committee_name'] = select_info_raw.split(':')[0].strip()
            select_info['role'] = select_info_raw.split(':')[1].strip()
            select_info['period'] = select_info_raw.split(':')[2].strip()
            select_appts[key] = select_info

In [171]:
select_appts

{0: {'duration': 'House Committee',
  'position': 'Member',
  'session': '13th Parliament, 2nd Session'},
 1: {'duration': 'House Committee',
  'position': 'Member',
  'session': '13th Parliament, 1st Session'}}

In [163]:
m_appts

{0: {'duration': '11 September 2015 to Current', 'position': 'Sembawang GRC'}}

In [160]:
o_appts

{0: {'duration': '1 May 2018 to Current',
  'position': 'Senior Parliamentary Secretary, Ministry of Home Affairs'},
 1: {'duration': '1 May 2018 to Current',
  'position': 'Senior Parliamentary Secretary, Ministry of Health'},
 2: {'duration': '1 May 2017 to 30 April 2018',
  'position': 'Parliamentary Secretary, Ministry of Health'},
 3: {'duration': '1 October 2015 to 30 April 2018',
  'position': 'Parliamentary Secretary, Ministry of Home Affairs'}}

In [407]:
## Build Profile
src = "https://www.parliament.gov.sg/mps/list-of-current-mps/mp/details/heng-swee-keat"
r = requests.get(src)
bs = BeautifulSoup(r.text,'html5lib')
prof = {}
prof['name'] = bs.find('h1',{'class':'indv-mp-header-mobile'}).get_text()
prof['designation'] = bs.find('div',{'class':'row mp-designation-wrap'}).find('div',{'class':"col-md-9 col-xs-12"}).get_text().split('\t',1)[0].replace('\n','')
prof['birth_year'] = re.sub('\n|\t','',bs.find_all('div',{'class':'row mp-party-wrap'})[1].find('div',{'class':'col-md-9 col-xs-12'}).get_text())
prof['constituency'] = bs.find('div', {'class':'row mp-constituency-wrap'}).find('div',{'class':"col-md-9 col-xs-12"}).get_text().split('\n')[1].strip()
prof['party'] = bs.find('div',{'class':'row mp-party-wrap'}).find('div',{'class':"col-md-9 col-xs-12"}).get_text().strip()
appts = bs.find('div',{'class':'indv-mp-info'})
appts_clean = re.sub('(<!-- .* -->)','<seperator>',str(appts))
for item in split_appts:
    if'Office-Holding Appointments' in item:
        #OFFICE HOLDING APPOINTMENTS
        office_appts = item
        office_appts_list = BeautifulSoup(office_appts,'html').find_all('div', {'class':'row mp-info'})
        o_appts = {}
        for key,item in enumerate(office_appts_list):
            pos_info = {}
            pos_info_raw = re.sub('\n\n','',re.sub('\n\n\n',' : ', item.get_text()))
            pos_info['duration'] = pos_info_raw.split(':')[0].strip()
            pos_info['position'] = pos_info_raw.split(':')[1].strip()
            o_appts[key] = pos_info
        prof['office_appts'] = o_appts
    
    if'Member of Parliament' in item:
        # MEMBER OF PARLIAMENT INFO
        mp_appts = item
        mp_appts_list = BeautifulSoup(mp_appts,'html').find_all('div', {'class':'row mp-info'})
        m_appts = {}
        for key,item in enumerate(mp_appts_list):
            mp_info = {}
            mp_info_raw = re.sub('\n\n','',re.sub('\n\n\n',' : ', item.get_text()))
            mp_info['duration'] = mp_info_raw.split(':')[0].strip()
            mp_info['position'] = mp_info_raw.split(':')[1].strip()
            m_appts[key] = mp_info 
        prof['m_appts'] = m_appts
         
    if 'Select Committees' in item:
        # SELECT COMMITEE
        select_list = BeautifulSoup(select,'html').find_all('div', {'class':'row mp-info'})
        select_appts = {}
        for key,item in enumerate(select_list):
            select_info = {}
            select_info_raw = re.sub('\n\n','',re.sub('\n\n\n',' : ', item.get_text()))
            select_info['committee_name'] = select_info_raw.split(':')[0].strip()
            select_info['role'] = select_info_raw.split(':')[1].strip()
            select_info['period'] = select_info_raw.split(':')[2].strip()
            select_appts[key] = select_info
        prof['select_appts'] = select_appts


In [408]:
prof

{'name': 'Mr Heng Swee Keat',
 'designation': 'Deputy Prime Minister and Minister for Finance',
 'birth_year': '1961',
 'constituency': 'Tampines GRC',
 'party': 'People’s Action Party',
 'office_appts': {0: {'duration': '1 May 2018 to Current',
   'position': 'Senior Parliamentary Secretary, Ministry of Home Affairs'},
  1: {'duration': '1 May 2018 to Current',
   'position': 'Senior Parliamentary Secretary, Ministry of Health'},
  2: {'duration': '1 May 2017 to 30 April 2018',
   'position': 'Parliamentary Secretary, Ministry of Health'},
  3: {'duration': '1 October 2015 to 30 April 2018',
   'position': 'Parliamentary Secretary, Ministry of Home Affairs'}},
 'm_appts': {0: {'duration': '11 September 2015 to Current',
   'position': 'Sembawang GRC'}},
 'select_appts': {0: {'committee_name': 'House Committee',
   'role': 'Member',
   'period': '13th Parliament, 2nd Session'},
  1: {'committee_name': 'House Committee',
   'role': 'Member',
   'period': '13th Parliament, 1st Session'}}

## Get list of MPs

In [178]:
list_url = 'https://www.parliament.gov.sg/mps/list-of-current-mps'
r_list = requests.get(list_url)
doc = BeautifulSoup(r_list.text)

In [187]:
listmps_raw = doc.find_all('div',{'class':'col-md-8 col-xs-12 mp-sort-name'})
mp_list = [ref.find('a')['href'] for ref in listmps_raw]

In [189]:
mp_list[:3]

['/mps/current-list-of-mps/mp/details/amrin-amin',
 '/mps/current-list-of-mps/mp/details/ang-hin-kee',
 '/mps/current-list-of-mps/mp/details/ang-wei-neng']

In [409]:
def get_info(uri,data):
    BASE_URL = "https://www.parliament.gov.sg"
    r = requests.get(BASE_URL + uri)
    bs = BeautifulSoup(r.text,'html5lib')
    prof = {}
    try:
        prof['name'] = bs.find('h1',{'class':'indv-mp-header-mobile'}).get_text()
        prof['designation'] = bs.find('div',{'class':'row mp-designation-wrap'}).find('div',{'class':"col-md-9 col-xs-12"}).get_text().split('\t',1)[0].replace('\n','')
        if prof['designation'] != 'Nominated Member of Parliament':
            prof['party'] = bs.find('div',{'class':'row mp-party-wrap'}).find('div',{'class':"col-md-9 col-xs-12"}).get_text().strip()
            if prof['designation'] != 'Non-Constituency Member of Parliament':
                prof['constituency'] = bs.find('div', {'class':'row mp-constituency-wrap'}).find('div',{'class':"col-md-9 col-xs-12"}).get_text().split('\n')[1].strip()
                prof['birth_year'] = re.sub('\n|\t','',bs.find_all('div',{'class':'row mp-party-wrap'})[1].find('div',{'class':'col-md-9 col-xs-12'}).get_text())
                appts = bs.find('div',{'class':'indv-mp-info'})
                appts_clean = re.sub('(<!-- .* -->)','<seperator>',str(appts))
                for item in split_appts:
                    if'Office-Holding Appointments' in item:
                        #OFFICE HOLDING APPOINTMENTS
                        office_appts = item
                        office_appts_list = BeautifulSoup(office_appts,'html').find_all('div', {'class':'row mp-info'})
                        o_appts = {}
                        for key,item in enumerate(office_appts_list):
                            pos_info = {}
                            pos_info_raw = re.sub('\n\n','',re.sub('\n\n\n',' : ', item.get_text()))
                            pos_info['duration'] = pos_info_raw.split(':')[0].strip()
                            pos_info['position'] = pos_info_raw.split(':')[1].strip()
                            o_appts[key] = pos_info
                        prof['office_appts'] = o_appts

                    if'Member of Parliament' in item:
                        # MEMBER OF PARLIAMENT INFO
                        mp_appts = item
                        mp_appts_list = BeautifulSoup(mp_appts,'html').find_all('div', {'class':'row mp-info'})
                        m_appts = {}
                        for key,item in enumerate(mp_appts_list):
                            mp_info = {}
                            mp_info_raw = re.sub('\n\n','',re.sub('\n\n\n',' : ', item.get_text()))
                            mp_info['duration'] = mp_info_raw.split(':')[0].strip()
                            mp_info['position'] = mp_info_raw.split(':')[1].strip()
                            m_appts[key] = mp_info 
                        prof['m_appts'] = m_appts

                    if 'Select Committees' in item:
                        # SELECT COMMITEE
                        select_list = BeautifulSoup(select,'html').find_all('div', {'class':'row mp-info'})
                        select_appts = {}
                        for key,item in enumerate(select_list):
                            select_info = {}
                            select_info_raw = re.sub('\n\n','',re.sub('\n\n\n',' : ', item.get_text()))
                            select_info['committee_name'] = select_info_raw.split(':')[0].strip()
                            select_info['role'] = select_info_raw.split(':')[1].strip()
                            select_info['period'] = select_info_raw.split(':')[2].strip()
                            select_appts[key] = select_info
                        prof['select_appts'] = select_appts
        else:
            prof['birth_year'] = re.sub('\n|\t','',bs.find_all('div',{'class':'row mp-party-wrap'})[0].find('div',{'class':'col-md-9 col-xs-12'}).get_text())
        data.append(prof)
        return data
    except:
        print(f"URL broke at {BASE_URL + uri}")

In [410]:
from tqdm import tqdm

mp = []
for uri in tqdm(mp_list):
    mp = get_info(uri,mp)
    




  0%|          | 0/100 [00:00<?, ?it/s]


  1%|          | 1/100 [00:00<00:28,  3.43it/s]


  2%|▏         | 2/100 [00:00<00:26,  3.71it/s]


  3%|▎         | 3/100 [00:00<00:24,  3.88it/s]


  4%|▍         | 4/100 [00:00<00:23,  4.07it/s]


  5%|▌         | 5/100 [00:01<00:22,  4.31it/s]


  6%|▌         | 6/100 [00:01<00:28,  3.29it/s]


  7%|▋         | 7/100 [00:01<00:28,  3.31it/s]


  8%|▊         | 8/100 [00:02<00:25,  3.67it/s]


  9%|▉         | 9/100 [00:02<00:22,  4.07it/s]


 10%|█         | 10/100 [00:02<00:20,  4.34it/s]


 11%|█         | 11/100 [00:02<00:19,  4.53it/s]


 12%|█▏        | 12/100 [00:02<00:21,  4.14it/s]


 13%|█▎        | 13/100 [00:03<00:20,  4.34it/s]


 14%|█▍        | 14/100 [00:03<00:18,  4.62it/s]


 15%|█▌        | 15/100 [00:03<00:19,  4.40it/s]


 16%|█▌        | 16/100 [00:03<00:17,  4.76it/s]


 17%|█▋        | 17/100 [00:05<00:42,  1.96it/s]


 18%|█▊        | 18/100 [00:05<00:34,  2.39it/s]


 19%|█▉        | 19/100 [00:05<00:27,  2.90it/

In [334]:
df_mp = pd.DataFrame(mp)

In [340]:
df_mp.head()

,birth_year,constituency,designation,m_appts,name,office_appts,party,select_appts,salutation
0,1978,Sembawang GRC,"Senior Parliamentary Secretary, Ministry of Ho...",{0: {'duration': '11 September 2015 to Current...,Amrin Amin,"{0: {'duration': '1 May 2018 to Current', 'pos...",People's Action Party,"{0: {'duration': 'House Committee', 'position'...",Mr
1,1965,Ang Mo Kio GRC,Member of Parliament,{0: {'duration': '11 September 2015 to Current...,Ang Hin Kee,"{0: {'duration': '1 May 2018 to Current', 'pos...",People’s Action Party,"{0: {'duration': 'House Committee', 'position'...",Mr
2,1967,Jurong GRC,Member of Parliament,{0: {'duration': '11 September 2015 to Current...,Ang Wei Neng,"{0: {'duration': '1 May 2018 to Current', 'pos...",People’s Action Party,"{0: {'duration': 'House Committee', 'position'...",Mr
3,1970,Tampines GRC,"Senior Parliamentary Secretary, Ministry of Tr...",{0: {'duration': '11 September 2015 to Current...,Baey Yam Keng,"{0: {'duration': '1 May 2018 to Current', 'pos...",People’s Action Party,"{0: {'duration': 'House Committee', 'position'...",Mr
4,1976,Fengshan,Member of Parliament,{0: {'duration': '11 September 2015 to Current...,Cheryl Chan Wei Ling,"{0: {'duration': '1 May 2018 to Current', 'pos...",People's Action Party,"{0: {'duration': 'House Committee', 'position'...",Miss


### Clean MP Data

In [336]:
## create salutation column and map it accordingly
df_mp['salutation'] = df_mp.name.map(lambda x : x.split()[0] if 'Assoc Prof' not in x else 'Assoc Prof')
df_mp['name'] = df_mp.name.map(lambda x : x.split(' ',maxsplit=1)[1] if 'Assoc Prof' not in x else x.split(' ',maxsplit=2)[2])


In [359]:
df_mp.birth_year.sort_values(ascending=False).tail()

23                                                 1941
35                                                  ...
24                                                  NaN
64                                                  NaN
77                                                  NaN
Name: birth_year, dtype: object

In [361]:
df_mp.iloc[35]

birth_year                                                    ...
constituency                                         Nee Soon GRC
designation                                  Member of Parliament
m_appts         {0: {'duration': '11 September 2015 to Current...
name                                        Kwek Hian Chuan Henry
office_appts    {0: {'duration': '1 May 2018 to Current', 'pos...
party                                       People's Action Party
select_appts    {0: {'duration': 'House Committee', 'position'...
salutation                                                     Mr
Name: 35, dtype: object

In [338]:
display(df_mp.iloc[0].m_appts)
display(df_mp.iloc[0].office_appts)
display(df_mp.iloc[0].select_appts)

{0: {'duration': '11 September 2015 to Current', 'position': 'Sembawang GRC'}}

{0: {'duration': '1 May 2018 to Current',
  'position': 'Senior Parliamentary Secretary, Ministry of Home Affairs'},
 1: {'duration': '1 May 2018 to Current',
  'position': 'Senior Parliamentary Secretary, Ministry of Health'},
 2: {'duration': '1 May 2017 to 30 April 2018',
  'position': 'Parliamentary Secretary, Ministry of Health'},
 3: {'duration': '1 October 2015 to 30 April 2018',
  'position': 'Parliamentary Secretary, Ministry of Home Affairs'}}

{0: {'duration': 'House Committee',
  'position': 'Member',
  'session': '13th Parliament, 2nd Session'},
 1: {'duration': 'House Committee',
  'position': 'Member',
  'session': '13th Parliament, 1st Session'}}

In [368]:
df_mp['birth_year'] = df_mp.birth_year.map(lambda x: np.nan if len(str(x))>5 else x)

In [369]:
df_mp.birth_year.sort_values(ascending=False).tail()

23    1941
24     NaN
35     NaN
64     NaN
77     NaN
Name: birth_year, dtype: object

In [378]:
df_mp.birth_year = df_mp.birth_year.map(lambda x : pd.to_datetime(x,format='%Y'))

In [394]:
df_mp['year']= df_mp.birth_year.map(lambda x : x.year)

In [612]:
df_mp.head()

,birth_year,constituency,designation,m_appts,name,office_appts,party,select_appts,salutation,year
0,1978-01-01,Sembawang GRC,"Senior Parliamentary Secretary, Ministry of Ho...",{0: {'duration': '11 September 2015 to Current...,Amrin Amin,"{0: {'duration': '1 May 2018 to Current', 'pos...",People's Action Party,"{0: {'duration': 'House Committee', 'position'...",Mr,1978.0
1,1965-01-01,Ang Mo Kio GRC,Member of Parliament,{0: {'duration': '11 September 2015 to Current...,Ang Hin Kee,"{0: {'duration': '1 May 2018 to Current', 'pos...",People’s Action Party,"{0: {'duration': 'House Committee', 'position'...",Mr,1965.0
2,1967-01-01,Jurong GRC,Member of Parliament,{0: {'duration': '11 September 2015 to Current...,Ang Wei Neng,"{0: {'duration': '1 May 2018 to Current', 'pos...",People’s Action Party,"{0: {'duration': 'House Committee', 'position'...",Mr,1967.0
3,1970-01-01,Tampines GRC,"Senior Parliamentary Secretary, Ministry of Tr...",{0: {'duration': '11 September 2015 to Current...,Baey Yam Keng,"{0: {'duration': '1 May 2018 to Current', 'pos...",People’s Action Party,"{0: {'duration': 'House Committee', 'position'...",Mr,1970.0
4,1976-01-01,Fengshan,Member of Parliament,{0: {'duration': '11 September 2015 to Current...,Cheryl Chan Wei Ling,"{0: {'duration': '1 May 2018 to Current', 'pos...",People's Action Party,"{0: {'duration': 'House Committee', 'position'...",Miss,1976.0


In [613]:
df_mp['party'] = df_mp.party.map(lambda x : re.sub("’","'",str(x)))

In [614]:
df_mp.party.value_counts()

People's Action Party    82
Workers' Party            9
nan                       9
Name: party, dtype: int64

## Save to CSV

In [615]:
df_mp.to_csv('current_mp_profile')

In [616]:
clean_mp.salutation.value_counts()

Mr            62
Ms            15
Dr            13
Prof           3
Assoc Prof     3
Miss           2
Er             1
Mrs            1
Name: salutation, dtype: int64

In [617]:
clean_mp[clean_mp['salutation'] == 'Er']

,name,designation,birth_year,party,constituency,salutation,matched_name
37,DR LEE BEE WAH,Member of Parliament,1960-01-01,People’s Action Party,Nee Soon GRC,Er,LEE BEE WAH


In [34]:
# remove words in brackets
template_name_party.Name = template_name_party.Name.map(lambda x : re.sub(r'\[[^()]*\]','',x))
# remove parentheses
template_name_party.Name = template_name_party.Name.map(lambda x : re.sub(r'\(|\)','',x))
# remove characters
template_name_party.Name = template_name_party.Name.map(lambda x : x.replace(',',' '))

NameError: name 'template_name_party' is not defined

In [ ]:
# reset index
template_name_party = template_name_party.reset_index(drop=True)
display(template_name_party)

In [35]:
template_name_party.Name = template_name_party.Name.map(lambda x : x.upper())

json_template = template_name_party.to_dict(orient='records')

NameError: name 'template_name_party' is not defined

In [592]:
json_template[:3]

[{'Name': 'ABBAS BIN ABU AMIN', 'Party': "People's Action Party"},
 {'Name': 'ABDUL AZIZ BIN KARIM', 'Party': "People's Action Party"},
 {'Name': 'ABDUL HAMID BIN JUMAT',
  'Party': 'United Malays National Organisation'}]

In [593]:
names = [name['Name'] for name in json_template]
names[:3]

['ABBAS BIN ABU AMIN', 'ABDUL AZIZ BIN KARIM', 'ABDUL HAMID BIN JUMAT']

In [594]:
template_name_party.Party.value_counts().head(10)

People's Action Party         318
Workers' Party                127
Independent                   106
Liberal Socialist Party        61
Barisan Sosialis               52
National Solidarity Party      40
Singapore Democratic Party     40
Singapore Alliance             39
United People's Party          38
Progressive Party              30
Name: Party, dtype: int64

In [618]:
clean_mp = df_mp[['name','designation','birth_year','party','constituency','salutation']]
clean_mp.head()

,name,designation,birth_year,party,constituency,salutation
0,Amrin Amin,"Senior Parliamentary Secretary, Ministry of Ho...",1978-01-01,People's Action Party,Sembawang GRC,Mr
1,Ang Hin Kee,Member of Parliament,1965-01-01,People's Action Party,Ang Mo Kio GRC,Mr
2,Ang Wei Neng,Member of Parliament,1967-01-01,People's Action Party,Jurong GRC,Mr
3,Baey Yam Keng,"Senior Parliamentary Secretary, Ministry of Tr...",1970-01-01,People's Action Party,Tampines GRC,Mr
4,Cheryl Chan Wei Ling,Member of Parliament,1976-01-01,People's Action Party,Fengshan,Miss


In [619]:
clean_mp.name = clean_mp.name.map(lambda x : x.upper())

In [620]:
clean_mp.head()

,name,designation,birth_year,party,constituency,salutation
0,AMRIN AMIN,"Senior Parliamentary Secretary, Ministry of Ho...",1978-01-01,People's Action Party,Sembawang GRC,Mr
1,ANG HIN KEE,Member of Parliament,1965-01-01,People's Action Party,Ang Mo Kio GRC,Mr
2,ANG WEI NENG,Member of Parliament,1967-01-01,People's Action Party,Jurong GRC,Mr
3,BAEY YAM KENG,"Senior Parliamentary Secretary, Ministry of Tr...",1970-01-01,People's Action Party,Tampines GRC,Mr
4,CHERYL CHAN WEI LING,Member of Parliament,1976-01-01,People's Action Party,Fengshan,Miss


Using Levenshtein distances to map similar names with the two DFs.

In [622]:
clean_mp[clean_mp['constituency'].isnull()]

,name,designation,birth_year,party,constituency,salutation
15,ARASU DURAISAMY,Nominated Member of Parliament,1968-01-01,nan,NaN,Mr
18,DOUGLAS FOO,Nominated Member of Parliament,1969-01-01,nan,NaN,Mr
24,DANIEL GOH PEI SIONG,Non-Constituency Member of Parliament,NaT,Workers' Party,NaN,Assoc Prof
27,TERENCE HO WEE SAN,Nominated Member of Parliament,1969-01-01,nan,NaN,Mr
44,LIM SUN SUN,Nominated Member of Parliament,1972-01-01,nan,NaN,Prof
52,MOHAMED IRSHAD,Nominated Member of Parliament,1989-01-01,nan,NaN,Mr
60,ANTHEA ONG,Nominated Member of Parliament,1968-01-01,nan,NaN,Ms
64,LEON PERERA,Non-Constituency Member of Parliament,NaT,Workers' Party,NaN,Mr
68,IRENE QUAY SIEW CHING,Nominated Member of Parliament,1975-01-01,nan,NaN,Ms
77,DENNIS TAN LIP FONG,Non-Constituency Member of Parliament,NaT,Workers' Party,NaN,Mr


In [569]:
from nltk.metrics import edit_distance, edit_distance_align

In [547]:
def get_closest_match_id(x, string_list):

    best_match = None
    scores = []
    threshold = 30
    for current_string in string_list:
        scores.append(edit_distance(x, current_string))

    min_score_idx = scores.index(min(scores))

    return string_list[min_score_idx]

In [630]:
clean_mp['matched_name'] = clean_mp[~clean_mp['constituency'].isnull()].name.map(lambda x : get_closest_match_id(x,names))

/Users/syamil/anaconda3/envs/pivot/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [626]:
get_closest_match_id('ALEX YAM ZIMING',names)

'BAEY YAM KENG'

In [627]:
edit_distance('MELVIN YONG YIK CHYE','YONG Yik Chye, Melvin')

20

## Export files for local editing

While the distance similarity seems to be able to map most of the data correctly, we will correct the labelling via manual edit since it is a short list.

In [631]:
clean_mp.to_csv('clean_mp.csv')

In [632]:
template_name_party.to_csv('refer.csv')

## Use fuzzy matching to match individuals

In [6]:
mp_list = pd.read_csv('clean_mp.csv',infer_datetime_format=True)

In [7]:
party_list = pd.read_csv('party.csv',infer_datetime_format=True)

In [8]:
mp_list.drop(columns=['Unnamed: 0','matched_name'],axis=1,inplace=True)

In [9]:
party_list.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [10]:
party_list_recent = party_list.groupby('Name').last()
party_list_recent.reset_index(inplace=True)

In [11]:
ncmp_list = pd.read_csv('ncmp.csv')
nmp_list = pd.read_csv('nmp.csv')

In [12]:
# remove words in brackets
party_list.Name = party_list.Name.map(lambda x : re.sub(r'\[[^()]*\]','',x))
# remove parentheses
party_list.Name = party_list.Name.map(lambda x : re.sub(r'\(|\)','',x))
# remove characters
party_list.Name = party_list.Name.map(lambda x : x.replace(',',' '))

In [17]:
# reset index
party_list = party_list.reset_index(drop=True)
display(party_list.head())

,Name,Party,GE,Constituency,Margin,election_type,election_year
0,ABBAS BIN ABU AMIN,People's Action Party,GE 1980,Pasir Panjang,72.6%,GE,1980
1,ABBAS BIN ABU AMIN,People's Action Party,GE 1984,Pasir Panjang,59.3%,GE,1984
2,ABBAS BIN ABU AMIN,People's Action Party,GE 1988,Pasir Panjang GRC,61.6%,GE,1988
3,ABDUL AZIZ BIN KARIM,People's Action Party,GE 1968,Kallang,Uncontested,GE,1968
4,ABDUL AZIZ BIN KARIM,People's Action Party,GE 1972,Kallang,79.8%,GE,1972


In [31]:
party_list.Name = party_list.Name.map(lambda x : x.upper())
party_list_recent.Name = party_list_recent.Name.map(lambda x : x.upper())

In [32]:
party_list_grouped = party_list.groupby('Name').count().reset_index().drop(columns=['GE',
                                                              'Constituency',
                                                              'Margin','election_type',
                                                              'election_year'])
party_list_grouped = party_list_grouped.rename(columns={'Party':'count_election'})

In [35]:
party_list_grouped.merge(party_list_recent,on='Name',how='inner')[['Name','count_election','Party']].to_csv('grouped_party_count')

In [47]:
party_list_grouped.to_csv('election_counts.csv')

## Load files

In [652]:
edited = pd.read_csv('clean_mp_train.csv',infer_datetime_format=True)
edited.head()

,Unnamed: 0,name,designation,birth_year,party,constituency,salutation,matched_name
0,0,AMRIN AMIN,"Senior Parliamentary Secretary, Ministry of Ho...",1978-01-01,People's Action Party,Sembawang GRC,Mr,AMRIN BIN AMIN
1,1,ANG HIN KEE,Member of Parliament,1965-01-01,People's Action Party,Ang Mo Kio GRC,Mr,ANG HIN KEE
2,2,ANG WEI NENG,Member of Parliament,1967-01-01,People's Action Party,Jurong GRC,Mr,ANG WEI NENG
3,3,BAEY YAM KENG,"Senior Parliamentary Secretary, Ministry of Tr...",1970-01-01,People's Action Party,Tampines GRC,Mr,BAEY YAM KENG
4,4,CHERYL CHAN WEI LING,Member of Parliament,1976-01-01,People's Action Party,Fengshan,Miss,CHAN WEI LING CHERYL


In [653]:
edited = edited.drop('Unnamed: 0',axis=1) # drop unnamed column

In [668]:
merged = edited.merge(template_name_party,how='right',left_on='matched_name',right_on='Name')
merged.head()

,name,designation,birth_year,party,constituency,salutation,matched_name,Name,Party
0,AMRIN AMIN,"Senior Parliamentary Secretary, Ministry of Ho...",1978-01-01,People's Action Party,Sembawang GRC,Mr,AMRIN BIN AMIN,AMRIN BIN AMIN,People's Action Party
1,ANG HIN KEE,Member of Parliament,1965-01-01,People's Action Party,Ang Mo Kio GRC,Mr,ANG HIN KEE,ANG HIN KEE,People's Action Party
2,ANG WEI NENG,Member of Parliament,1967-01-01,People's Action Party,Jurong GRC,Mr,ANG WEI NENG,ANG WEI NENG,People's Action Party
3,BAEY YAM KENG,"Senior Parliamentary Secretary, Ministry of Tr...",1970-01-01,People's Action Party,Tampines GRC,Mr,BAEY YAM KENG,BAEY YAM KENG,People's Action Party
4,CHERYL CHAN WEI LING,Member of Parliament,1976-01-01,People's Action Party,Fengshan,Miss,CHAN WEI LING CHERYL,CHAN WEI LING CHERYL,People's Action Party


In [670]:
merged = merged[['Name','Party','designation','birth_year','constituency','salutation']]

In [671]:
merged.birth_year = merged.birth_year.map(lambda x : pd.to_datetime(x))

## Insert names to DB

In [674]:
merged = merged.rename({'Name':'name','Party':'party'},axis='columns')
merged.head()

,name,party,designation,birth_year,constituency,salutation
0,AMRIN BIN AMIN,People's Action Party,"Senior Parliamentary Secretary, Ministry of Ho...",1978-01-01,Sembawang GRC,Mr
1,ANG HIN KEE,People's Action Party,Member of Parliament,1965-01-01,Ang Mo Kio GRC,Mr
2,ANG WEI NENG,People's Action Party,Member of Parliament,1967-01-01,Jurong GRC,Mr
3,BAEY YAM KENG,People's Action Party,"Senior Parliamentary Secretary, Ministry of Tr...",1970-01-01,Tampines GRC,Mr
4,CHAN WEI LING CHERYL,People's Action Party,Member of Parliament,1976-01-01,Fengshan,Miss


In [675]:
from db.models import postgres_engine

merged.to_sql('mem_parliament',postgres_engine,if_exists='append',index=False)